This file is made to build logic and see data in a run time environment. 
use this file to learn how it was created and which objects are passed overtime through out the 
run Time

In [ ]:
import pandas as pd
import find_all_path
import numpy as np

In [ ]:
nodes_list, long_key_number , shortest_key_number , longest_path, shortest_path = find_all_path.get_paths('CVE-2022-20644','CVE-2022-20788')

In [ ]:
#creating a nodes converting tuple to list 
# unnecessary could be done without using it
nodes_list = [list(i) for i in nodes_list]


In [ ]:
# creating a list of path name that will be used to create dataframe for getting tuple to dictionary output
path = [f'path_{i}' for i in range(len(longest_path))]

In [ ]:
df = pd.DataFrame(nodes_list,columns=path)
df.head(3)

In [ ]:
df.columns

In [ ]:
df.loc[df['path_3']=='attack_001','path_4'].drop_duplicates()

In [ ]:
df.loc[df['path_3'] == 'attack_002','path_4'].drop_duplicates().values.tolist()

In [ ]:
for i in path:
    print(f"{i} = {df[i].drop_duplicates().values.tolist()}")

In [ ]:
dictionary = {}
for i in range(len(path)-1):
    unique_value_for_path = df[path[i]].drop_duplicates().values.tolist()
    for j in unique_value_for_path:
        connections_from_current_path = df.loc[df[path[i]]==j,path[i+1]].drop_duplicates().values.tolist()
        if None in connections_from_current_path:
            for _ in range(connections_from_current_path.count(None)):
                connections_from_current_path.remove(None)
        # print(j,connections_from_current_path)
        if j is not None:
            if j not in dictionary:
                dictionary[j] = connections_from_current_path
            else:
                dictionary[j] += connections_from_current_path

for i in dictionary:
    print(f'{i} length = {len(dictionary[i])} = {dictionary[i]}')

# BACK TRACKING

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(user = 'root', password = 'root', host = '127.0.0.1', database='attack_graph_v2')
mycursor = mydb.cursor()

In [ ]:
mycursor.execute("select vulnerability.cve_id from vulnerability where vulnerability.asset_type = (SELECT assets.asset_type from assets WHERE assets.asset_id = 'a7') and vulnerability.cve_id in(SELECT postcondition.cve_id from postcondition where postcondition.gained_access ='high');")
mycursor.fetchall()

in asset back tracking

In [ ]:
asset_id = 'A7'
current_access = 'high'
in_asset_back_tracking_list = []
mycursor.execute(f"select precondition.required_access, precondition.vulnerability from precondition where precondition.vulnerability in (select vulnerability.cve_id from vulnerability where vulnerability.asset_type = (SELECT assets.asset_type from assets WHERE assets.asset_id = '{asset_id}') and vulnerability.cve_id in(SELECT postcondition.cve_id from postcondition where postcondition.gained_access ='{current_access}')) and precondition.connection_type is NULL;")
for i in mycursor.fetchall():
    in_asset_back_tracking_list.append([f'{asset_id}.{i[0]}',f'{asset_id}.{i[1]}'])
in_asset_back_tracking_list

change the output to A7.low , A7,CVE-2009-4030

# out of asset back tracking

In [ ]:
mycursor.execute(f"select * from precondition where precondition.vulnerability in (select vulnerability.cve_id from vulnerability where vulnerability.asset_type = (SELECT assets.asset_type from assets WHERE assets.asset_id = '{asset_id}') and vulnerability.cve_id in(SELECT postcondition.cve_id from postcondition where postcondition.gained_access ='{current_access}')) and precondition.connection_type is not  NULL;")
out_asset_back_tracking_list = []
for i in mycursor.fetchall():
    mycursor.execute(f"SELECT connections.source_asset from connections where connections.destination_asset = '{asset_id}' and connections.connection_type = '{i[1]}';")
    for j in mycursor.fetchall():
        if i[0] == None:
            out_asset_back_tracking_list.append([f'{j[0]}.high',f'{j[0]}.{i[1]},{asset_id}',f'{asset_id}.{i[2]}'])
            out_asset_back_tracking_list.append([f'{j[0]}.low',f'{j[0]}.{i[1]},{asset_id}',f'{asset_id}.{i[2]}'])
        else:
            out_asset_back_tracking_list.append([f'{j[0]}.{i[0]}',f'{j[0]}.{i[1]},{asset_id}',f'{asset_id}.{i[2]}'])
out_asset_back_tracking_list

LONE TESTING

In [ ]:
mycursor.execute(f"select * from precondition where precondition.vulnerability in (select vulnerability.cve_id from vulnerability where vulnerability.asset_type = (SELECT assets.asset_type from assets WHERE assets.asset_id = '{asset_id}') and vulnerability.cve_id in(SELECT postcondition.cve_id from postcondition where postcondition.gained_access ='{current_access}')) and precondition.connection_type is NULL;")
i =  mycursor.fetchall()
i[0][1] == None


In [ ]:
a = 'A1.low'
c , d = a.split('.')
print(c,d)

In [ ]:
import serverconnection_v2

x = serverconnection_v2.MYSQL_connection()

In [ ]:
x.find_out_of_asset_vulnerabilities('A7','high')

In [ ]:
def find_back_track_path(asset_id,current_access,travelled_path):
    if f'{asset_id}.{current_access}' in travelled_path:
        print('ending',travelled_path)
        return
    travelled_path += (f'{asset_id}.{current_access}',)
    print('travelling : ',travelled_path)
    # In asset Back tracking tree
    for i in x.in_asset_back_tracking(asset_id,current_access):
        path = travelled_path + (i,)
        asset_id_in,current_access_in = i[0].split('.')
        find_back_track_path(asset_id_in,current_access_in,path)
    
    # Out asset Back Tracking tree
    for j in x.out_of_asset_back_tracking(asset_id,current_access):
        path = travelled_path + (j,)
        asset_id_out,current_access_out = j[0].split('.')
        find_back_track_path(asset_id_out,current_access_out,path)

travelled_path_1 = ()
find_back_track_path('A7','high',travelled_path_1)

In [ ]:
travelled_path_1

In [ ]:


class BackTracking:
    def __init__(self):
        self.main_path = []
        self.x = serverconnection_v2.MYSQL_connection()
    
    def find_back_track_path(self,asset_id,current_access,travelled_path):
    


back_tracking = BackTracking()
x = back_tracking.get_back_back_tracked('A7','high')
print(f'THE BACK TRACKED PATH THAT ARE SAVED \n {x}')

# *_creation of the CRITICALITY_*

In [ ]:
from serverconnection_v2 import MYSQL_connection
from typing import Union

server = MYSQL_connection()

# Node creation

In [ ]:
class Node:
    #node is simple object that hold the data which are needed
    def __init__(self,node_id : str, access_level : str):
        
        self.node_id = node_id
        self.node_type = server.find_asset_type(node_id)
        self.node_importance_score = server.show_asset_importance_score(node_id)
        self.access_level = access_level
        self.distance_from_start = 9999

    def __repr__(self) -> str:
        return f'{self.node_id} + {self.access_level}'
    
    def give_id(self):
        return f'{self.node_id}.{self.access_level}'

    def give_description(self):
        return f'Node ID = {self.node_id}Node_type = {self.node_type}Node Importance Score in the Network = {self.node_importance_score}Node distance from start = {self.distance_from_start}'


class Vulnerability_node:
    def __init__(self,from_asset : Union[Node , str],to_asset : Union[Node , str],vulnerability_id : str,connection_type: str = None):
        self.node_name = f'{str(from_asset)}+{str(to_asset)}+{vulnerability_id}'
        self.from_asset = from_asset
        self.to_asset = to_asset
        self.connection_type = connection_type
        self.vulnerability_id = vulnerability_id
        self.vulnerabilty_score = server.find_score(to_asset.node_id,vulnerability_id)
        self.distance_from_start = 9999
        self.impact_score = None
        self.ciricality_score = None


    def __repr__(self) -> str:
        if self.connection_type == None:
            return str({'Attack type': 'IN ASSET ATTACK','from':str(self.from_asset),'to':str(self.to_asset),'vulnerability used':self.vulnerability_id})
        else:
            return str({'Attack type': 'OUT OF ASSET ATTACK','from':str(self.from_asset),'to':str(self.to_asset),'vulnerability used':self.vulnerability_id,'Connection':self.connection_type})

    def give_id(self):
        return f'{self.from_asset.give_id()}.{self.to_asset.give_id()}.{self.vulnerability_id}'
    
    def calculate_ciriticality(self):
        pass

    def give_description(self):
        return f'From asset = {self.from_asset.node_id}TO asset = {self.to_asset.node_id}Node distance from start = {self.distance_from_start}'

# Making atomic NODE OBJECT

In [ ]:
class NodeList:
    def __init__(self):
        self.asset_nodes = []
        self.vulnerability_nodes = []

    def create_asset_node(self,asset_id:str,access_level: str):
        for i in self.asset_nodes:
            if i.node_id == asset_id and i.access_level == access_level:
                return i
        else:
            some_variable = Node(node_id=asset_id,access_level=access_level)
            self.asset_nodes.append(some_variable)
            return some_variable
    
    def create_vulnerability_node(self, from_asset: Union[Node, str], to_asset: Union[Node, str], vulnerability_id: str,connection_type: str = None):
        for i in self.vulnerability_nodes:
            if i.from_asset == from_asset and i.to_asset == to_asset and i.vulnerability_id == vulnerability_id and i.connection_type == connection_type:
                return i
        else:
            some_variable =  Vulnerability_node(from_asset=from_asset,
                                        to_asset=to_asset,
                                        vulnerability_id=vulnerability_id,
                                        connection_type=connection_type)
            self.vulnerability_nodes.append(some_variable)
            return some_variable

# finding path with OBJECT IN OBJECT MODEL

In [ ]:
class Find_ciritical_path:
    def __init__(self) -> None:
        self.myserver = MYSQL_connection()
        self.main_list = []
        self.node = NodeList()

    def check_if_in(self, node: Node, path: Union[tuple, list]):
        for i in path:
            if str(node) == str(i):
                return True
        else:
            return False
    
    def set_distance_from_start(self,node : Union[Node, Vulnerability_node], distance : int):

        if node.distance_from_start > distance:
            node.distance_from_start = distance
        

    def find_ciritical_path(self, current_node: Node, goal_node: Node, travelling_path: tuple, distance : int = 0):
        if self.check_if_in(current_node, travelling_path):
            # print('done')
            # self.main_list.append(travelling_path[:-1])
            return
        else:
            self.set_distance_from_start(current_node , distance= distance + 1)
            if current_node == goal_node:
                travelling_path += (current_node,)
                print('here')
                self.main_list.append(travelling_path)
                return
            travelling_path +=(current_node,)
            print(travelling_path)
            for i in self.myserver.find_in_asset_vulnerability(current_node.node_id, current_node.access_level):
                # print('here',i)
                asset_id_in, current_access_in = self.myserver.find_post_condition(i[1])
                # print(asset_id_in,current_access_in)
                new_node = self.node.create_asset_node(asset_id_in, current_access_in)

                vulnerability_id = i[1].split('.')[1]
                
                vulnerability_node = self.node.create_vulnerability_node(current_node, new_node, vulnerability_id)
                self.set_distance_from_start(vulnerability_node, distance + 2)

                path = travelling_path + (vulnerability_node,)
                # print(new_node,goal_node,path)
                self.find_ciritical_path(new_node, goal_node, path,distance=distance+2)

            for j in self.myserver.find_out_of_asset_vulnerabilities(current_node.node_id, current_node.access_level):
                
                # reference what the data is in J
                # [['A1.high', 'A1.remote_access.A3', 'A3.CVE-2019-12749']]
                # kindly follow and update the upper data to get the 
                asset_id_out, current_access_out = self.myserver.find_post_condition(j[2])
                # print(asset_id_out,current_access_out)
                new_node = self.node.create_asset_node(asset_id_out, current_access_out)
                vulnerability_id = j[2].split('.')[1]
                connection_type = j[1].split('.')[1]
                vulnerability_node = self.node.create_vulnerability_node(current_node, new_node,
                                                            vulnerability_id=vulnerability_id,
                                                            connection_type=connection_type)
                self.set_distance_from_start(vulnerability_node,distance + 2)
                path = travelling_path + (vulnerability_node,)
                self.find_ciritical_path(new_node, goal_node, path,distance=distance+2)

    def get_critical_path(self, asset_id, current_access, goal_asset_id, goal_access_level):
        empty_tuple = ()
        starting_node = self.node.create_asset_node(asset_id, current_access)
        goal_node = self.node.create_asset_node(goal_asset_id, goal_access_level)

        self.find_ciritical_path(current_node=starting_node, goal_node=goal_node, travelling_path=empty_tuple)
        print('FOUND PATHS = {}\n'.format(len(self.main_list)))
        for i in self.main_list:
            print(i,end='\n****************** *************\n')
    
    def get_unique_nodes(self):
        return self.node.asset_nodes

In [ ]:
x = Find_ciritical_path()
x.get_critical_path('A1','low','A7','high')

In [ ]:
for i in x.main_list:
    for j in i:
        print(j.distance_from_start,end = ',')
    print()

```CREATING ADJACENCYMATRIX```

In [ ]:
adjacency_dictionary = {}
unique_nodes = x.node.asset_nodes + x.node.vulnerability_nodes
for i in unique_nodes:
    print(i)
for i in unique_nodes:
    empty_list = []
    for j in range(len(x.main_list)):
        for k in range(len(x.main_list[j])-1):
            if i == x.main_list[j][k]:
                if x.main_list[j][k+1] not in empty_list:
                    empty_list.append(x.main_list[j][k+1])
    adjacency_dictionary[i] = empty_list

adjacency_dictionary

<h2>adjacency matrix to JSON NODE</h2>

In [ ]:
class Json_node_builder:
    
    def __init__(self) -> None:
        self.parent_count = 0
        self.parent_node_dict = {}
    
    def list_to_node(self,adjacency_matrix):
        nodes = []
        for i in adjacency_matrix:

            # give_id gives unique names to node can be used for connection also
            nodes.append({'data':{'id':f'{i.give_id()}',
                                'name':f'{i.give_id()}',
                                'description':f'{i.give_description()}'}})
        connection = 0
        for i in adjacency_matrix:
            for j in adjacency_matrix[i]:
                nodes.append({'data' : {'id' : f'X{connection}',
                                        'source' : i.give_id(),
                                        'target' : j.give_id(),
                                        'directed' : 'true'}})
                connection +=1
        
        return nodes

In [ ]:
something_object = Json_node_builder()

something_object.list_to_node(adjacency_dictionary)

In [ ]:
from shortest_path_finding import BFS_SP

print(len(BFS_SP(adjacency_dictionary,unique_nodes[4],unique_nodes[1])))

In [ ]:
isinstance(unique_nodes[2],Vulnerability_node)

In [ ]:
a = []
a.append(Node('A7','low'))
a.append(Node('A6','low'))
a.append(Node('A53','high'))
print(a[1])



In [ ]:
node = Node('A7','Windows 7',9)
str(node) in a

# TESTING CLASS

In [ ]:
class Xyz:
    def __init__(self,name):
        self.name = name

a = []

x = Xyz('ron')
a.append(x)
x = Xyz('sumon')
a.append(x)
x = Xyz('vamshi')
a.append(x)

In [ ]:
class Test:
    def __init__(self,value) -> None:
        self.value = value

In [ ]:
x = Test('BISHWAJIT')
y = Test('Vamshi')

print(x,y)

In [ ]:
x.name = 'something'
y.putki = 'something'
print(x,y)

In [ ]:
print(x.value,x.name)

In [ ]:
dic = {}
dic[x] = 'Here is a data'

In [ ]:
dic[x]

In [ ]:
import criticality_path_finder
x = criticality_path_finder.Find_ciritical_path()
x = x.get_critical_path('A1', 'low', 'A7', 'high')

In [ ]:
print(x)

In [ ]:
x = "{'asset': 'A1', 'asset_type': 'Windows 7', 'asset_importance_score': 3}"
x = x.replace("'",'"')
import json
x = json.loads(x)
x

In [ ]:
sum = 0 
for letter in list("Close") : 
    sum += ord(letter) 
    print(sum)



In [ ]:
sum = []

In [ ]:
import numpy
print(numpy.cumsum( 5 *[ord(letter) for letter in list("Close") ]))

In [ ]:
def func(a,b):
    if a == b:
        return 0
    if a > b:
        return 1+func(a-1,b)
    if b > a:
        return 1+func(a+1,b-1)
    else:
        return 3+func(a,b)
    return 1

func(5,1)


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello_world():
    return "<p>Hello, World!</p>"

if __name__ == '__main__':
    app.run(host='0.0.0.0',debug = True)

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(host='localhost',user = 'ron',password = 'root',database = 'attack_graph_v2')
mycursor = mydb.cursor()

In [ ]:
import pandas as pd

df = pd.read_csv('Datas/Att&ckToCveMappings.csv')
df.head(5)

In [ ]:
cve_data = df['CVE ID'].values.tolist()
len(cve_data),cve_data

In [ ]:
cve_data = list(set(cve_data))[:20]

In [ ]:
mycursor.execute('select DISTINCT vulnerability.asset_type from vulnerability')
types_of_asset = [i[0] for i in mycursor.fetchall()]
types_of_asset

In [ ]:
# types_of_asset += ['Fedora 14','Debian 16','Mongodb 4.2']
# types_of_asset

In [ ]:
import random
insertion_data = []
for i in cve_data:
    insertion_data.append([random.choice(types_of_asset),i,round(random.uniform(0.0,10.0),1)])

insertion_data[0:20]
# len(insertion_data)


In [ ]:
mycursor.executemany('insert into vulnerability values (%s,%s,%s)',insertion_data)
mycursor.execute('select * from vulnerability')
mycursor.fetchall()

In [ ]:
mydb.commit()

In [ ]:
assets_to_insert = []
for i in range(11):
    assets_to_insert.append([f'A{i+15}',random.choice(types_of_asset),random.randint(1,10)])
assets_to_insert

In [ ]:
mycursor.executemany('insert into assets values (%s,%s,%s)',assets_to_insert)
mycursor.execute('select * from assets')
x = mycursor.fetchall()
len(x),x

In [ ]:
mydb.commit()

In [ ]:
# mycursor.execute(f'insert into postcondition values ("{cve_data[0]}","high")')
mydb.rollback()

In [ ]:
for i in cve_data:
    mycursor.execute(f'insert into postcondition values ("{i}","{random.choice(["high","low"])}")')
    # mydb.commit()

mycursor.execute('select * from postcondition')
len(mycursor.fetchall())

In [ ]:
mydb.commit()

In [ ]:
pre_condition_data_insertion = []
for i in cve_data:
    
    mycursor.execute('insert into precondition values (%s,%s,%s)',(random.choice(['High','Low',None]),random.choice(['remote_access','network_access',None]),i))

mycursor.execute('select * from precondition')
len(mycursor.fetchall())

In [ ]:
mydb.commit()

In [ ]:
mycursor.execute('select asset_id from assets')
assets = [i[0] for i in mycursor.fetchall()]
assets

In [ ]:
assets = list(assets)

In [ ]:

for i in range(200):
    source = random.choice(assets)
    destination = random.choice([asset for asset in assets if asset != source])
    connection = random.choice(['remote_access','network_access'])

    mycursor.execute('insert into connections values (%s,%s,%s)',(source,destination,connection))

mycursor.execute('select * from connections')
len(mycursor.fetchall())

In [ ]:
mydb.commit()

In [ ]:
mydb

In [ ]:
x = [ i for i in range(10)]
print(x)

In [ ]:
x[-1:-4:-1]

In [ ]:
import hashlib

with open('paths_checking.txt') as f:
    lines = f.readlines()

hash = []

for i in lines:
    hash.append(hashlib.sha256(i.encode()).hexdigest())

len(hash),len(set(hash))

In [ ]:
len(hash)

In [ ]:
len(set(hash))

In [ ]:
import json

In [ ]:
string_1 = '''{"nodes":[{"ip":"10.0.0.2","mac":"5e:9b:c3:ff:26:dc","type":"Host","id":"H1"},{"ip":"10.0.0.1","mac":"82:c8:40:03:27:ef","type":"Host","id":"H2"},{"ip":"10.0.0.3","mac":"a6:52:bd:65:22:ab","type":"Host","id":"H3"},{"type":"Switch","id":"S1"},{"type":"Switch","id":"S2"},{"type":"Switch","id":"S3"}],"links":[{"source":"S2","target":"S1","ports":{"S2":"s2-eth1","S1":"s1-eth1"}},{"source":"S1","target":"S2","ports":{"S1":"s1-eth1","S2":"s2-eth1"}},{"source":"S3","target":"S2","ports":{"S3":"s3-eth1","S2":"s2-eth2"}},{"source":"S2","target":"S3","ports":{"S2":"s2-eth2","S3":"s3-eth1"}},{"source":"H1","target":"S2","ports":{"H1":"s2-eth3"}},{"source":"S2","target":"H1","ports":{"S2":"s2-eth3"}},{"source":"H2","target":"S1","ports":{"H2":"s1-eth2"}},{"source":"S1","target":"H2","ports":{"S1":"s1-eth2"}},{"source":"H3","target":"S3","ports":{"H3":"s3-eth2"}},{"source":"S3","target":"H3","ports":{"S3":"s3-eth2"}}],"flow-tables":{"S1":[{"priority":65535,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":1260,"duration_sec":19,"duration_nsec":311000000,"packet_count":21,"length":96,"flags":0,"actions":["OUTPUT:CONTROLLER"],"match":{"dl_dst":"01:80:c2:00:00:0e","dl_type":35020},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":238,"duration_sec":16,"duration_nsec":142000000,"packet_count":3,"length":104,"flags":0,"actions":["OUTPUT:2"],"match":{"in_port":1,"dl_src":"5e:9b:c3:ff:26:dc","dl_dst":"82:c8:40:03:27:ef"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":140,"duration_sec":16,"duration_nsec":122000000,"packet_count":2,"length":104,"flags":0,"actions":["OUTPUT:1"],"match":{"in_port":2,"dl_src":"82:c8:40:03:27:ef","dl_dst":"5e:9b:c3:ff:26:dc"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":238,"duration_sec":16,"duration_nsec":63000000,"packet_count":3,"length":104,"flags":0,"actions":["OUTPUT:2"],"match":{"in_port":1,"dl_src":"a6:52:bd:65:22:ab","dl_dst":"82:c8:40:03:27:ef"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":140,"duration_sec":16,"duration_nsec":58000000,"packet_count":2,"length":104,"flags":0,"actions":["OUTPUT:1"],"match":{"in_port":2,"dl_src":"82:c8:40:03:27:ef","dl_dst":"a6:52:bd:65:22:ab"},"table_id":0},{"priority":0,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":8138,"duration_sec":19,"duration_nsec":338000000,"packet_count":66,"length":80,"flags":0,"actions":["OUTPUT:CONTROLLER"],"match":{},"table_id":0}],"S2":[{"priority":65535,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":2520,"duration_sec":19,"duration_nsec":415000000,"packet_count":42,"length":96,"flags":0,"actions":["OUTPUT:CONTROLLER"],"match":{"dl_dst":"01:80:c2:00:00:0e","dl_type":35020},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":238,"duration_sec":16,"duration_nsec":335000000,"packet_count":3,"length":104,"flags":0,"actions":["OUTPUT:1"],"match":{"in_port":3,"dl_src":"5e:9b:c3:ff:26:dc","dl_dst":"82:c8:40:03:27:ef"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":140,"duration_sec":16,"duration_nsec":296000000,"packet_count":2,"length":104,"flags":0,"actions":["OUTPUT:3"],"match":{"in_port":1,"dl_src":"82:c8:40:03:27:ef","dl_dst":"5e:9b:c3:ff:26:dc"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":238,"duration_sec":16,"duration_nsec":254000000,"packet_count":3,"length":104,"flags":0,"actions":["OUTPUT:1"],"match":{"in_port":2,"dl_src":"a6:52:bd:65:22:ab","dl_dst":"82:c8:40:03:27:ef"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":140,"duration_sec":16,"duration_nsec":240000000,"packet_count":2,"length":104,"flags":0,"actions":["OUTPUT:2"],"match":{"in_port":1,"dl_src":"82:c8:40:03:27:ef","dl_dst":"a6:52:bd:65:22:ab"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":238,"duration_sec":16,"duration_nsec":196000000,"packet_count":3,"length":104,"flags":0,"actions":["OUTPUT:3"],"match":{"in_port":2,"dl_src":"a6:52:bd:65:22:ab","dl_dst":"5e:9b:c3:ff:26:dc"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":140,"duration_sec":16,"duration_nsec":187000000,"packet_count":2,"length":104,"flags":0,"actions":["OUTPUT:2"],"match":{"in_port":3,"dl_src":"5e:9b:c3:ff:26:dc","dl_dst":"a6:52:bd:65:22:ab"},"table_id":0},{"priority":0,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":8278,"duration_sec":19,"duration_nsec":435000000,"packet_count":68,"length":80,"flags":0,"actions":["OUTPUT:CONTROLLER"],"match":{},"table_id":0}],"S3":[{"priority":65535,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":1320,"duration_sec":19,"duration_nsec":603000000,"packet_count":22,"length":96,"flags":0,"actions":["OUTPUT:CONTROLLER"],"match":{"dl_dst":"01:80:c2:00:00:0e","dl_type":35020},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":238,"duration_sec":16,"duration_nsec":484000000,"packet_count":3,"length":104,"flags":0,"actions":["OUTPUT:1"],"match":{"in_port":2,"dl_src":"a6:52:bd:65:22:ab","dl_dst":"82:c8:40:03:27:ef"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":140,"duration_sec":16,"duration_nsec":460000000,"packet_count":2,"length":104,"flags":0,"actions":["OUTPUT:2"],"match":{"in_port":1,"dl_src":"82:c8:40:03:27:ef","dl_dst":"a6:52:bd:65:22:ab"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":238,"duration_sec":16,"duration_nsec":424000000,"packet_count":3,"length":104,"flags":0,"actions":["OUTPUT:1"],"match":{"in_port":2,"dl_src":"a6:52:bd:65:22:ab","dl_dst":"5e:9b:c3:ff:26:dc"},"table_id":0},{"priority":1,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":140,"duration_sec":16,"duration_nsec":405000000,"packet_count":2,"length":104,"flags":0,"actions":["OUTPUT:2"],"match":{"in_port":1,"dl_src":"5e:9b:c3:ff:26:dc","dl_dst":"a6:52:bd:65:22:ab"},"table_id":0},{"priority":0,"cookie":0,"idle_timeout":0,"hard_timeout":0,"byte_count":7966,"duration_sec":19,"duration_nsec":616000000,"packet_count":64,"length":80,"flags":0,"actions":["OUTPUT:CONTROLLER"],"match":{},"table_id":0}]}}'''
string_1

In [ ]:
x = json.loads(string_1)

In [ ]:
print(x)

In [ ]:
for i in x:
    print(i)

In [ ]:
for i in x['nodes']:
    print(i)

In [ ]:
from criticality_path_finder import Find_ciritical_path

x = Find_ciritical_path()

x.get_critical_path('A1', 'low', 'A12', 'high')

In [ ]:
for j in sorted(x.adjacency_dictionary,key=lambda i:len(x.adjacency_dictionary[i]),reverse=True)[:3]:
    print(type(j))

In [ ]:
three_most_connected_nodes = sorted(x.adjacency_dictionary,key=lambda j:len(x.adjacency_dictionary[j]),reverse=True)[:3]

vulnerabilities = []
for i in x.adjacency_dictionary:
    if any(item in three_most_connected_nodes for item in x.adjacency_dictionary[i]):
        vulnerabilities.append(i)

for i in vulnerabilities:
    print(i.vulnerabilty_score, end = ' >>> ')
vulnerabilities.sort(key= lambda x: x.vulnerabilty_score,reverse=True)
print()
for i in vulnerabilities:
    print(i.vulnerabilty_score, end = ' >>> ')

# _Creating simple finder using object in object model_

In [ ]:
import criticality_path_finder

In [ ]:
class Find_attack_range:
    def __init__(self) -> None:
        self.myserver = criticality_path_finder.MYSQL_connection()
        self.main_list = []
        self.node = criticality_path_finder.NodeList()
        self.adjacency_dictionary = {}
        self.unique_nodes = []
        self.number = 0

    def find_all_attackable_path(self,current_node : criticality_path_finder.Node, travelling_path : tuple):
        if current_node in travelling_path:
            if travelling_path[:-1] not in self.main_list:
                self.main_list.append(travelling_path[:-1])
            return
        else:
            # print(travelling_path)
            travelling_path += (current_node,)
            for i in self.myserver.find_in_asset_vulnerability(current_node.node_id, current_node.access_level):
                asset_id_in, current_access_in = self.myserver.find_post_condition(i[1])
                # print(asset_id_in,current_access_in)
                new_node = self.node.create_asset_node(asset_id_in, current_access_in)

                vulnerability_id = i[1].split('.')[1]
                
                vulnerability_node = self.node.create_vulnerability_node(current_node, new_node, vulnerability_id)
                # self.set_distance_from_start(vulnerability_node, distance + 2)

                path = travelling_path + (vulnerability_node,)
                # print(new_node,goal_node,path)
                self.find_all_attackable_path(new_node,path)
            
            for j in self.myserver.find_out_of_asset_vulnerabilities(current_node.node_id, current_node.access_level):
                
                # reference what the data is in J
                # [['A1.high', 'A1.remote_access.A3', 'A3.CVE-2019-12749']]
                # kindly follow and update the upper data to get the 
                asset_id_out, current_access_out = self.myserver.find_post_condition(j[2])
                # print(asset_id_out,current_access_out)
                new_node = self.node.create_asset_node(asset_id_out, current_access_out)
                vulnerability_id = j[2].split('.')[1]
                connection_type = j[1].split('.')[1]
                vulnerability_node = self.node.create_vulnerability_node(current_node, new_node,
                                                            vulnerability_id=vulnerability_id,
                                                            connection_type=connection_type)
                # self.set_distance_from_start(vulnerability_node,distance + 2)
                path = travelling_path + (vulnerability_node,)
                self.find_all_attackable_path(new_node, path)

    def get_unique_nodes(self):
        return self.node.asset_nodes
    
    def list_to_adjacency_matrix(self):
        self.adjacency_dictionary = {}
        self.unique_nodes = []
        self.unique_nodes = self.node.asset_nodes + self.node.vulnerability_nodes
        # for i in self.unique_nodes:
        #     print(i)
        for i in self.unique_nodes:
            empty_list = []
            for j in range(len(self.main_list)):
                for k in range(len(self.main_list[j])-1):
                    if i == self.main_list[j][k]:
                        if self.main_list[j][k+1] not in empty_list:
                            empty_list.append(self.main_list[j][k+1])
            self.adjacency_dictionary[i] = empty_list


    def get_all_path(self,asset_id,current_access):
        empty_tuple = ()
        starting_node = self.node.create_asset_node(asset_id, current_access)
        # goal_node = self.node.create_asset_node(goal_asset_id, goal_access_level)
        # self.node.goal_asset_importance_score = goal_node.node_importance_score
        # 1. First its trying to find the path by using network
        # while finding path it also create nodes with value ** distance from start
        self.find_all_attackable_path(current_node=starting_node, travelling_path=empty_tuple)
        if len(self.main_list) == 0:

            # if no path is found then return 0 which will return an error page

            print(f'NO path FOUND for {starting_node} to {goal_node}')
            return 0

        print('PATHS FOUND ',len(self.main_list))
        print(self.main_list)
        print('time taken ', self.number)

        # 2. creating adjacency matrix from 2D list
        self.list_to_adjacency_matrix()

        # 3. calculate shortest distance to goal

        # self.calculate_shortest_path_to_target(goal_node=goal_node)

        # 4. now calculate the criticality score and impact score 
        # for i in self.node.vulnerability_nodes:
        #     i.calculate_criticality()

        three_most_connected_nodes = sorted(self.adjacency_dictionary,key=lambda x:len(self.adjacency_dictionary[x]),reverse=True)[:3]
        most_vulnerabilities = []
        for i in self.adjacency_dictionary:
            if any(item in three_most_connected_nodes for item in self.adjacency_dictionary[i]):
                most_vulnerabilities.append(i)

        most_vulnerabilities.sort(key= lambda x: x.vulnerabilty_score,reverse=True)

        # three_most_connected_nodes[0].color = '#FF0000'
        # three_most_connected_nodes[1].color = '#FF5733'
        # three_most_connected_nodes[2].color = '#FFA07A'
        
        most_vulnerabilities[0].color = '#FF0000'
        most_vulnerabilities[1].color = '#FF5733'
        try:
            most_vulnerabilities[2].color = '#FFA07A'
        except:
            pass
        safe_nodes =[starting_node]
        temp_nodes = list(self.adjacency_dictionary.keys()) # strong temp nodes
        for i in temp_nodes:
            if len(self.adjacency_dictionary[i]) == 0 and i not in safe_nodes:
                self.adjacency_dictionary.pop(i)
        # 4. create JSON FROM THE DATA AND RETURN IT TO ROUTE.PY
        json_converter = criticality_path_finder.Json_node_builder()
        return json_converter.list_to_node(self.adjacency_dictionary)

In [ ]:
x = Find_attack_range()
x.get_all_path('A1','low')

In [1]:
import criticality_path_finder
x = criticality_path_finder.Find_ciritical_path()
json_return = x.get_critical_path('A1','low','A14','high')

PATHS FOUND  108
[(A1 + low, {'Attack type': 'IN ASSET ATTACK', 'from': 'A1 + low', 'to': 'A1 + high', 'vulnerability used': 'CVE-2019-1468'}, A1 + high, {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A1 + high', 'to': 'A3 + high', 'vulnerability used': 'CVE-2019-12749', 'Connection': 'remote_access'}, A3 + high, {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A3 + high', 'to': 'A7 + high', 'vulnerability used': 'CVE-2006-3486', 'Connection': 'network_access'}, A7 + high, {'Attack type': 'IN ASSET ATTACK', 'from': 'A7 + high', 'to': 'A7 + low', 'vulnerability used': 'CVE-2020-13125'}, A7 + low, {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A7 + low', 'to': 'A12 + high', 'vulnerability used': 'CVE-2012-2319', 'Connection': 'network_access'}, A12 + high, {'Attack type': 'IN ASSET ATTACK', 'from': 'A12 + high', 'to': 'A12 + low', 'vulnerability used': 'CVE-2016-1409'}, A12 + low, {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A12 + low', 'to': 'A13 + high', 'vulnerability used': 'CVE-

TypeError: calculate_shortest_path_to_target() got an unexpected keyword argument 'goal_node'

In [3]:
x.adjacency_dictionary

{A1 + low: [{'Attack type': 'IN ASSET ATTACK', 'from': 'A1 + low', 'to': 'A1 + high', 'vulnerability used': 'CVE-2019-1468'},
  {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A1 + low', 'to': 'A3 + high', 'vulnerability used': 'CVE-2019-12749', 'Connection': 'remote_access'}],
 A14 + high: [],
 A1 + high: [{'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A1 + high', 'to': 'A3 + high', 'vulnerability used': 'CVE-2019-12749', 'Connection': 'remote_access'}],
 A3 + high: [{'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A3 + high', 'to': 'A7 + high', 'vulnerability used': 'CVE-2006-3486', 'Connection': 'network_access'}],
 A7 + high: [{'Attack type': 'IN ASSET ATTACK', 'from': 'A7 + high', 'to': 'A7 + low', 'vulnerability used': 'CVE-2020-13125'},
  {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A7 + high', 'to': 'A12 + high', 'vulnerability used': 'CVE-2012-2319', 'Connection': 'network_access'},
  {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A7 + high', 'to': 'A12 + low', 'vulnerabil

In [5]:
from shortest_path_finding import BFS_SP

starting_node = x.node.create_asset_node('A1','low')
ending_node = x.node.create_asset_node('A14','high')

variable = BFS_SP(x.adjacency_dictionary,starting_node,ending_node)

Shortest path =  A1 + low {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A1 + low', 'to': 'A3 + high', 'vulnerability used': 'CVE-2019-12749', 'Connection': 'remote_access'} A3 + high {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A3 + high', 'to': 'A7 + high', 'vulnerability used': 'CVE-2006-3486', 'Connection': 'network_access'} A7 + high {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A7 + high', 'to': 'A12 + high', 'vulnerability used': 'CVE-2012-2319', 'Connection': 'network_access'} A12 + high {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A12 + high', 'to': 'A13 + high', 'vulnerability used': 'CVE-2004-1371', 'Connection': 'remote_access'} A13 + high {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A13 + high', 'to': 'A14 + high', 'vulnerability used': 'CVE-2019-1817', 'Connection': 'remote_access'} A14 + high


In [8]:
variable,json_return

([A1 + low,
  {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A1 + low', 'to': 'A3 + high', 'vulnerability used': 'CVE-2019-12749', 'Connection': 'remote_access'},
  A3 + high,
  {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A3 + high', 'to': 'A7 + high', 'vulnerability used': 'CVE-2006-3486', 'Connection': 'network_access'},
  A7 + high,
  {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A7 + high', 'to': 'A12 + high', 'vulnerability used': 'CVE-2012-2319', 'Connection': 'network_access'},
  A12 + high,
  {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A12 + high', 'to': 'A13 + high', 'vulnerability used': 'CVE-2004-1371', 'Connection': 'remote_access'},
  A13 + high,
  {'Attack type': 'OUT OF ASSET ATTACK', 'from': 'A13 + high', 'to': 'A14 + high', 'vulnerability used': 'CVE-2019-1817', 'Connection': 'remote_access'},
  A14 + high],
 [{'data': {'id': 'A1.low',
    'name': 'A1.low',
    'description': 'Node ID = A1Node_type = Windows 7Node Importance Score in the Network = 3Node dist

In [12]:
for i in range(0,len(variable)-1):
    for j in json_return:
        try:
            if j['data']['source'] == variable[i].give_id() and j['data']['target'] == variable[i+1].give_id():
                j['css'] = {'line-color': 'red'}
        except:
            pass
json_return

[{'data': {'id': 'A1.low',
   'name': 'A1.low',
   'description': 'Node ID = A1Node_type = Windows 7Node Importance Score in the Network = 3Node distance from start = 1',
   'table_data': {'Asset Type': 'Windows 7', 'Access Level': 'low'}},
  'css': {'shape': 'circle', 'background-color': 'yellow'}},
 {'data': {'id': 'A14.high',
   'name': 'A14.high',
   'description': 'Node ID = A14Node_type = MySQL 5.3Node Importance Score in the Network = 9Node distance from start = 11',
   'table_data': {'Asset Type': 'MySQL 5.3', 'Access Level': 'high'}},
  'css': {'shape': 'circle', 'background-color': 'yellow'}},
 {'data': {'id': 'A1.high',
   'name': 'A1.high',
   'description': 'Node ID = A1Node_type = Windows 7Node Importance Score in the Network = 3Node distance from start = 3',
   'table_data': {'Asset Type': 'Windows 7', 'Access Level': 'high'}},
  'css': {'shape': 'circle', 'background-color': 'yellow'}},
 {'data': {'id': 'A3.high',
   'name': 'A3.high',
   'description': 'Node ID = A3Nod